In [1]:
import numpy as np
import pandas as pd

In [2]:
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D 
from keras.preprocessing.text import Tokenizer

In [3]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download("punkt")
from nltk.probability import FreqDist

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Николай\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [25]:
import warnings
warnings.filterwarnings('ignore')

In [26]:
import gensim

In [5]:
from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import re

## 1) структурирование датафрейма

In [7]:
df_row = pd.read_csv("leto.csv")
df_row = df_row.rename(columns = {'Rating':'class', 'Content':'text'})
df_train = df_row.drop(columns=['Date'])

df_train["class"] = np.where(df_train["class"] >= 4, 1, 0)
df_test = df_train.iloc[0:4000]
df_test = df_test[['text', 'class']]
df_train = df_train[['text', 'class']]

df_train = df_train.dropna()
df_test = df_test.dropna()

df_train.head()

,text,class
0,It just works!,1
1,В целом удобноное приложение...из минусов хотя...,1
2,Отлично все,1
3,Стал зависать на 1% работы антивируса. Дальше ...,1
4,"Очень удобно, работает быстро.",1


## 2) Предобработка

In [8]:
sw = set(get_stop_words("ru"))
exclude = set(punctuation)
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("\sне", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    return " ".join(txt)

df_train['text'] = df_train['text'].apply(preprocess_text)
df_test['text'] = df_test['text'].apply(preprocess_text)

In [9]:
train_corpus = " ".join(df_train["text"])
train_corpus = train_corpus.lower()

In [10]:
tokens = word_tokenize(train_corpus)
tokens_filtered = [word for word in tokens if word.isalnum()]

In [11]:
max_words = 100
dist = FreqDist(tokens_filtered)
tokens_filtered_top = [pair[0] for pair in dist.most_common(max_words-1)]
tokens_filtered_top[:10]

['приложение',
 'удобно',
 'работать',
 'удобный',
 'отлично',
 'нравиться',
 'хороший',
 'отличный',
 'телефон',
 'супер']

In [12]:
vocabulary = {v: k for k, v in dict(enumerate(tokens_filtered_top, 1)).items()}
vocabulary

{'приложение': 1,
 'удобно': 2,
 'работать': 3,
 'удобный': 4,
 'отлично': 5,
 'нравиться': 6,
 'хороший': 7,
 'отличный': 8,
 'телефон': 9,
 'супер': 10,
 'быстро': 11,
 'обновление': 12,
 'мочь': 13,
 'пароль': 14,
 'антивирус': 15,
 'пользоваться': 16,
 'банк': 17,
 'сбербанк': 18,
 'устраивать': 19,
 'вход': 20,
 'раз': 21,
 'карта': 22,
 'прошивка': 23,
 'рута': 24,
 'проблема': 25,
 'программа': 26,
 'писать': 27,
 'сделать': 28,
 'счёт': 29,
 'перевод': 30,
 'разработчик': 31,
 'приходиться': 32,
 'деньга': 33,
 'норма': 34,
 'ошибка': 35,
 'вводить': 36,
 'нормально': 37,
 'платёж': 38,
 'постоянно': 39,
 'около': 40,
 'довольный': 41,
 'исправить': 42,
 'понятно': 43,
 'смс': 44,
 'код': 45,
 'функция': 46,
 'свой': 47,
 'шаблон': 48,
 'вылетать': 49,
 'последний': 50,
 'зайти': 51,
 'мобильный': 52,
 'возможность': 53,
 'стать': 54,
 'право': 55,
 'приходить': 56,
 'делать': 57,
 'иня': 58,
 'установить': 59,
 'класс': 60,
 'заходить': 61,
 'проверка': 62,
 'root': 63,
 'опер

In [13]:
def text_to_sequence(text, maxlen):
    result = []
    tokens = word_tokenize(text.lower())
    tokens_filtered = [word for word in tokens if word.isalnum()]
    for word in tokens_filtered:
        if word in vocabulary:
            result.append(vocabulary[word])
    padding = [0]*(maxlen-len(result))
    return padding + result[-maxlen:]

In [14]:
max_len = 30
x_train = np.asarray([text_to_sequence(text, max_len) for text in df_train["text"]], dtype=np.int32)
x_test = np.asarray([text_to_sequence(text, max_len) for text in df_test["text"]], dtype=np.int32)

## 3) Keras без предобучения

In [15]:
num_classes = 2
y_train = keras.utils.to_categorical(df_train["class"], num_classes)
y_test = keras.utils.to_categorical(df_test["class"], num_classes)

In [16]:
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_len))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [17]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [18]:
epochs = 10
batch_size = 300

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1)

Epoch 1/10
62/62 [==============================] - 2s 20ms/step - loss: 0.3646 - accuracy: 0.8557 - val_loss: 0.3380 - val_accuracy: 0.8591
Epoch 2/10
62/62 [==============================] - 1s 18ms/step - loss: 0.2496 - accuracy: 0.8940 - val_loss: 0.3414 - val_accuracy: 0.8621
Epoch 3/10
62/62 [==============================] - 1s 17ms/step - loss: 0.2440 - accuracy: 0.8965 - val_loss: 0.3320 - val_accuracy: 0.8664
Epoch 4/10
62/62 [==============================] - 1s 18ms/step - loss: 0.2421 - accuracy: 0.8977 - val_loss: 0.3257 - val_accuracy: 0.8659
Epoch 5/10
62/62 [==============================] - 1s 17ms/step - loss: 0.2385 - accuracy: 0.8995 - val_loss: 0.3343 - val_accuracy: 0.8679
Epoch 6/10
62/62 [==============================] - 1s 17ms/step - loss: 0.2356 - accuracy: 0.9009 - val_loss: 0.3337 - val_accuracy: 0.8693
Epoch 7/10
62/62 [==============================] - 1s 16ms/step - loss: 0.2329 - accuracy: 0.9020 - val_loss: 0.3379 - val_accuracy: 0.8669
Epoch 8/10
62

## 3) Keras с предобучением

In [19]:
# 2Гб весов
vectors = gensim.models.KeyedVectors.load_word2vec_format('boost.vec') .vectors

In [20]:
matrix = [vectors[i][:128] for i in range(100)]

In [21]:
initial = keras.initializers.Constant(matrix)

In [22]:
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=128, embeddings_initializer=initial, input_length=max_len))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [23]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [24]:
epochs = 20
batch_size = 100

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1)

Epoch 1/20
186/186 [==============================] - 2s 7ms/step - loss: 0.3068 - accuracy: 0.8732 - val_loss: 0.3574 - val_accuracy: 0.8504
Epoch 2/20
186/186 [==============================] - 1s 6ms/step - loss: 0.2500 - accuracy: 0.8922 - val_loss: 0.3333 - val_accuracy: 0.8630
Epoch 3/20
186/186 [==============================] - 1s 6ms/step - loss: 0.2420 - accuracy: 0.8967 - val_loss: 0.3394 - val_accuracy: 0.8587
Epoch 4/20
186/186 [==============================] - 1s 7ms/step - loss: 0.2389 - accuracy: 0.8991 - val_loss: 0.3544 - val_accuracy: 0.8577
Epoch 5/20
186/186 [==============================] - 1s 7ms/step - loss: 0.2328 - accuracy: 0.9013 - val_loss: 0.3344 - val_accuracy: 0.8616
Epoch 6/20
186/186 [==============================] - 1s 7ms/step - loss: 0.2270 - accuracy: 0.9046 - val_loss: 0.3256 - val_accuracy: 0.8664
Epoch 7/20
186/186 [==============================] - 1s 7ms/step - loss: 0.2211 - accuracy: 0.9105 - val_loss: 0.3345 - val_accuracy: 0.8572
Epoch 